In [1]:
import pandas as pd
import psycopg2
import numpy as np



from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC



from tqdm import tqdm
tqdm.pandas()
import time



from bs4 import BeautifulSoup
import re

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = r"../chromedriver_win32/chromedriver.exe",chrome_options = chrome_options)

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\3931627501.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r"../chromedriver_win32/chromedriver.exe",chrome_options = chrome_options)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\3931627501.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = r"../chromedriver_win32/chromedriver.exe",chrome_options = chrome_options)


In [3]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

client_ids=['138','135','133','132','130','136','131','141','134','128','142','140','356','391','392','432','137']
client_id=[138,135,133,132,130,136,131,141,134,128,142,140,356,391,392,432,137]


tlds=['https://www.amazon.sg/dp/',
     'https://www.amazon.nl/dp/',
     'https://www.amazon.it/dp/',
     'https://www.amazon.in/dp/',
     'https://www.amazon.fr/dp/',
     'https://www.amazon.es/dp/',
     'https://www.amazon.de/dp/',
     'https://www.amazon.com/dp/',
     'https://www.amazon.com.mx/dp/',
     'https://www.amazon.com.au/dp/',
     'https://www.amazon.co.uk/dp/',
     'https://www.amazon.ae/dp/',
     'https://www.amazon.pl/dp/',
     'https://www.amazon.se/dp/',
     'https://www.amazon.com.br/dp/',
     'https://www.amazon.ca/dp/',
     'https://www.amazon.com.be/dp/']


tld_df=pd.DataFrame(list(zip(client_id,tlds)),columns=['client_id','tld'])


job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

query = f"""
select client_id, keyword, brand, title, price, sku, crawl_time, sponsored, channel_id, rating, rating_count,job_name
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])


data_df=serp_details[['client_id','sku','title','brand']]

data_df=data_df.drop_duplicates('sku')

query_1 = "select country, id from client_resource.client_master "
country_data = pd.read_sql_query(query_1, conn[1])

df1= pd.merge(data_df,country_data, left_on='client_id', right_on='id', how= 'left')


query_2 = f""" 
Select channel_sku_id,country_code, brand_name, categories,category_id
from entity.product_master
where channel_sku_id in {tuple(df1['sku'].dropna())} 
"""

pm_data = pd.read_sql_query(query_2, conn[1])

mapped_df=pd.merge(df1,pm_data, how='left', left_on=['sku','country'],right_on=['channel_sku_id','country_code'])

df2=mapped_df[mapped_df['categories'].isnull()][['client_id','country','sku','title','brand_name' ,"categories","category_id"]]
df3=pd.merge(df2,tld_df,on= 'client_id', how= 'left')
df3['product_link']=df3['tld']+df3['sku']
df4=df3.drop_duplicates(subset=['sku','country'])
df5=df4.reset_index()
df5['brand_name']=""

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '1.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

Enter keystone ids with comma.64e5ec9d31675555b916e40c,64e5eaa831675555b916e33e
 Enter no of days for which data needs brand mapping.
5


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\588336937.py:87: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  serp_details = pd.read_sql_query(query, conn[1])
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\588336937.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  country_data = pd.read_sql_query(query_1, conn[1])
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\588336937.py:106: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pm_data = pd.read_sql_query(query_2, conn[1])


In [4]:
for i in tqdm(range(0,len(df5))):  #len(df5)
    
    if df5['brand_name'][i]=="":
        
        try:
            print(i)
            timeout = 10
            locator = "imgTagWrapperId"
            link=df5['product_link'][i]
            try:


                print(link)
                driver.get(link)
                WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, locator)))
                brand=""

                try:

                    BrandVal = driver.find_element(By.ID,'bylineInfo')
                    brand = BrandVal.text
                    print(BrandVal.text)

                except:
                    try:
                        try:

                            bs=BeautifulSoup(driver.page_source, "html.parser")
                            imagebrand=bs.find('div',id="brandBar_feature_div")
                            try:

                                hidden_brand=imagebrand.find("img")['alt']

                            except:

                                hidden_brand=imagebrand.text

                            brand=hidden_brand
                            print(brand)



                        except:

                            bs=BeautifulSoup(driver.page_source, "html.parser")
                            brand=bs.find_all('div',class_="a-column a-span6 a-span-last a-text-right")[0].text.strip()
                            print(brand)
                    except:

                        Table2 = driver.find_element(By.ID,'productOverview_feature_div')
                        vals = Table2.find_elements(By.TAG_NAME,'td')

                        for v in range(len(vals)):
                            if vals[v].text in [ "Brand", "Marca" ]:
                                brand = vals[v+1].text
                                print(vals[v+1].text)
                df5['brand_name'][i]=brand


                try:
                    bread=driver.find_element(By.ID,"wayfinding-breadcrumbs_feature_div")
                    cats=[]
                    cat_ids=[]

                    links=bread.find_elements(By.TAG_NAME,'a')

                    for lnk in links:
                        cats.append(lnk.text)
                        url = lnk.get_attribute("href")

                        match = re.search(r"node=(\d+)", url)
                        node_id = match.group(1)
                        cat_ids.append(node_id)  
                    print(cats)



                except:      
                    cats=[]
                    cat_ids=[]
                    print("no cat found")            

                df5['categories'][i]=cats
                df5['category_id'][i]=cat_ids

            except:
                pass
        except:
            pass

        
#all language
df5['brand_name'] = df5['brand_name'].str.replace('^Visita la tienda de\s','').str.replace('^Visita Lo Store Di\s','').str.replace('^Brand:\s','').str.replace('\sStore','').str.replace('^Visit\sthe\s','').str.replace('^Marca:\s','').str.replace('^Visita\slo\sdi\s','').str.replace('^Visita\sla\sde\s','').str.replace('^Merk:\s','').str.replace('^De\s',''). str.replace('openen','').str.replace('^Marque :\s','').str.replace('^Visiter la boutique\s','')


df5['channel_id']=1
final_df=df5[['channel_id','sku','title','brand_name','country','categories','category_id']]
final_df.rename(columns={'sku':'channel_sku_id','country':'country_code'},inplace=True)

jobs_name=list(serp_details['job_name'].unique())



z=(list(serp_details['job_name'].unique()))
pattern = r'\d+'
requests=[]
# Loop over each string in the list and extract the numbers
for string in z:
    numbers = re.findall(pattern, string)
    requests.append(numbers)
rno = [item for inner_list in requests for item in inner_list]
file='brand_mapping_amazon {}.xlsx'.format(rno)

final_df.to_excel(file)

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['nitin.bhayana@1digitalstack.ai','ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s brand mapping of amazon. Please check brand mapping for following jobs.\n{}'.format(jobs_name)
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()

  0%|                                                                                         | 0/1550 [00:00<?, ?it/s]

0
https://www.amazon.in/dp/B09VT8QHMF


  0%|                                                                               | 1/1550 [00:11<4:52:17, 11.32s/it]

1
https://www.amazon.in/dp/B08KTYVHLW
Brand: Allen Solly


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\3096989248.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['brand_name'][i]=brand
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\3096989248.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['categories'][i]=cats
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\3096989248.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['catego

...

Brand: Sensodyne
['Health & Personal Care', 'Oral Care', 'Toothpastes']
1549
https://www.amazon.in/dp/B0C6JZCW64


100%|████████████████████████████████████████████████████████████████████████████| 1550/1550 [1:48:50<00:00,  4.21s/it]

Brand: Close Up
['Health & Personal Care', 'Oral Care', 'Toothpastes']



C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\3096989248.py:93: FutureWarning: The default value of regex will change from True to False in a future version.
  df5['brand_name'] = df5['brand_name'].str.replace('^Visita la tienda de\s','').str.replace('^Visita Lo Store Di\s','').str.replace('^Brand:\s','').str.replace('\sStore','').str.replace('^Visit\sthe\s','').str.replace('^Marca:\s','').str.replace('^Visita\slo\sdi\s','').str.replace('^Visita\sla\sde\s','').str.replace('^Merk:\s','').str.replace('^De\s',''). str.replace('openen','').str.replace('^Marque :\s','').str.replace('^Visiter la boutique\s','')
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_156016\3096989248.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'sku':'channel_sku_id','country':'

(221, b'2.0.0 Bye')

In [5]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

In [6]:
email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s brand mapping of amazon. Please check brand mapping for following jobs.\n{}'.format(jobs_name)
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()

(221, b'2.0.0 Bye')

In [7]:
    # try:
    #     rating=bs.find('div',class_="RRBox").text
    #     print("rating_count:" ,rating)
    #     df['rating_count'][i]=rating
    # except:
    #     df['rating_count'][i]=""  